In [ ]:
from bs4 import BeautifulSoup
import requests
import uuid
import json
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter variáveis de ambiente necessárias para a API Azure
azureai_endpoint = os.getenv("AZURE_ENDPOINT")
API_KEY = os.getenv("AZURE_OPENAI_KEY")

# Variáveis de ambiente a serem definidas no arquivo .env
# AZURE_OPENAI_KEY=sua_chave_openai
# AZURE_ENDPOINT=seu_endpoint_openai

# Link atualizado para a variável url
url = "https://www.example.com/artigo-de-exemplo"  # Substitua pelo seu link desejado

def extract_text(url):
    """
    Extrai o texto de uma página da web, removendo scripts e estilos.

    Args:
        url (str): URL da página da qual o texto será extraído.

    Returns:
        str: Texto extraído da página ou None se a solicitação falhar.
    """
    response = requests.get(url)
    if response.status_code == 200:
        # Utiliza BeautifulSoup para processar o HTML da página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove elementos script e style para limpar o conteúdo
        for script in soup(["script", "style"]):
            script.decompose()

        # Extrai o texto limpo da página
        text = soup.get_text(" ", strip=True)
        return text
    else:
        print("Falha ao buscar a URL. Código de status:", response.status_code)
        return None

# Testando a função de extração de texto
text = extract_text(url)

def translate_article(text, lang):
    """
    Traduz o texto usando a API da Azure OpenAI.

    Args:
        text (str): O texto a ser traduzido.
        lang (str): O idioma para o qual o texto deve ser traduzido.

    Returns:
        str: Texto traduzido em formato Markdown.
    """
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }

    # Estrutura da carga útil da solicitação, configurando a tradução
    payload = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "Você atua como tradutor de textos"
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"traduza: {text} para o idioma {lang} e responda apenas com a tradução no formato markdown"
                    }
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 900
    }

    # Ponto de extremidade da API Azure
    ENDPOINT = azureai_endpoint

    try:
        # Enviar a solicitação de tradução
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Levanta um erro se o status da resposta for inválido
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Retorna o conteúdo traduzido do JSON da resposta
    return response.json()['choices'][0]['message']['content']

# Exemplo de uso da função de tradução
if text:
    translated_text = translate_article(text, "fr")  # Traduzindo para francês como exemplo
    print(translated_text)
